<a href="https://colab.research.google.com/github/vitaldb/plans/blob/main/predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PlanS Model for predicting hepatocellular carcinoma in hepatitis B patients

In [2]:
!pip install scikit-learn==1.1.3
!pip install scikit-survival
!wget https://github.com/vitaldb/plans/raw/main/model.pkl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2023-01-29 17:44:39--  https://github.com/vitaldb/plans/raw/main/model.pkl
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/vitaldb/plans/main/model.pkl [following]
--2023-01-29 17:44:39--  https://raw.githubusercontent.com/vitaldb/plans/main/model.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4117320 (3.9M) [application/octet-stream]
Saving to: ‘model.pkl.2’

model.pkl.2         100%[=========

In [5]:
from sksurv.ensemble import RandomSurvivalForest
import numpy as np
import sys
import pickle

# load model
model = pickle.load(open('model.pkl', 'rb'))

# sample values
male = 0
e_t = 1  # entecavir (ETV) -> 0, tenofovir (TDF) -> 1
lc = 1  # None for missing value
plt = 172
tbil = 1.1
alt = 122
dna = 74279943
hbeag = 1

x = np.array([male, e_t, lc, plt, tbil, alt, dna, hbeag]).astype(float)
x = x[None, ...]

y_pred = model.predict(x).flatten()[0]
y_harz = 1 - model.predict_survival_function(x, return_array=True).flatten()

x[:, 1] = 0
y_pred_e = model.predict(x).flatten()[0]

x[:, 1] = 1
y_pred_t = model.predict(x).flatten()[0]

print(f'risk score = {y_pred:.3f}')
print(f'risk score (entecavir) = {y_pred_e:.3f}')
print(f'risk score (tenofovir) = {y_pred_t:.3f}')
print(model.event_times_)
print(y_harz)

risk score = 9.070
risk score (entecavir) = 9.400
risk score (tenofovir) = 9.070
[ 12.  13.  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.
  26.  27.  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.
  40.  41.  42.  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.
  54.  55.  56.  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.
  68.  69.  70.  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.
  82.  83.  84.  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  96.
  97.  98.  99. 100. 101. 102. 103. 104. 105. 107. 112. 113. 114. 116.
 118. 120. 127. 129. 131. 132. 133. 135. 148.]
[0.00218782 0.00517285 0.0066933  0.00871022 0.00906271 0.011664
 0.01383949 0.01490512 0.01817112 0.02006301 0.02099782 0.0225609
 0.02386307 0.02489127 0.02526941 0.02714035 0.02946329 0.02975815
 0.03152193 0.03276408 0.03406524 0.03463525 0.03540657 0.03604776
 0.03731364 0.03853516 0.03906297 0.04005766 0.04200235 0.04343221
 0.04464765 0.0471581  0.0506898  0.0508561